In [1]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [2]:
import pandas as pd 
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
DATA_DIR.exists()

True

In [3]:
dataset = DATA_DIR / 'reviews.csv'
dataset.exists()

True

In [5]:
df  = pd.read_csv(dataset)
df['score'].dropna(inplace=True)
df.head()

,uid,profile,anime_uid,text,score,scores,link
0,255938,DesolatePsyche,34096,\n \n \n \n ...,8,"{'Overall': '8', 'Story': '8', 'Animation': '8...",https://myanimelist.net/reviews.php?id=255938
1,259117,baekbeans,34599,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=259117
2,253664,skrn,28891,\n \n \n \n ...,7,"{'Overall': '7', 'Story': '7', 'Animation': '9...",https://myanimelist.net/reviews.php?id=253664
3,8254,edgewalker00,2904,\n \n \n \n ...,9,"{'Overall': '9', 'Story': '9', 'Animation': '9...",https://myanimelist.net/reviews.php?id=8254
4,291149,aManOfCulture99,4181,\n \n \n \n ...,10,"{'Overall': '10', 'Story': '10', 'Animation': ...",https://myanimelist.net/reviews.php?id=291149


In [7]:
df['score'] = df['score'].clip(lower=0, upper=10)

In [8]:
df.score.max(), df.score.min()

(10, 0)

In [11]:
reader = Reader(rating_scale=(0,10))
data = Dataset.load_from_df(df[['uid', 'anime_uid', 'score']], reader)

In [14]:
algo = SVD(verbose=True, n_epochs=20)
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([1.65024934, 1.64349445, 1.63473651, 1.62489238]),
 'test_mae': array([1.14336219, 1.13837987, 1.13106374, 1.12081455]),
 'fit_time': (7.206730604171753,
  7.340581178665161,
  7.196240186691284,
  7.258624315261841),
 'test_time': (0.39319419860839844,
  0.2613048553466797,
  0.2471911907196045,
  0.44959020614624023)}

In [15]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [19]:
testset = trainset.build_testset()
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)
#accuracy.mae(predictions, verbose=True)

RMSE: 0.7633


0.7633372714578521

In [21]:
sample_row = df.sample(n=1)
userId = sample_row['uid'].values[0]
animeId = sample_row['anime_uid'].values[0]
print(userId,animeId)

198933 28497


In [22]:
pred = algo.predict(uid=userId, iid=animeId)
pred.est

7.590695644166916

In [23]:
import pickle

In [24]:
algo_data = {"model": algo}
with open('model.pk1', 'wb') as f:
    pickle.dump(algo_data, f)

In [25]:
model_algo = None
with open('model.pk1', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [27]:
model_algo.predict(uid=userId, iid=animeId).est

7.590695644166916

In [28]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['uid']
    animeId = row['anime_uid']
    pred = model_algo.predict(uid=userId, iid=animeId).est
    print(userId,animeId,pred)

67073 20 8.87956037260727
14971 1604 9.873934162751068
258673 12355 9.796307757820685
53591 5081 2.370811123196175
130673 10029 5.139794946501107
244387 12189 7.144742510150106
263026 16049 8.819133530280611
250974 31658 8.05241692752753
247778 71 6.059469186897249
758 1827 8.773768777816107
